In [ ]:
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 23.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ"
# To use a different branch, change revision
# For example: revision="latest"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {prompt} ASSISTANT:

'''

print("\n\n*** Generate:")




*** Generate:


In [ ]:

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

<s> A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: Tell me about AI ASSISTANT:

Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that would typically require human intelligence, such as learning, problem-solving, perception, and decision-making. AI is a rapidly growing field that has the potential to transform many industries and improve people's lives in a variety of ways.

There are different types of AI, including:

1. Reactive machines: These AI systems are designed to react to specific situations based on pre-defined rules. They do not have the ability to learn from past experiences or make decisions on their own.

2. Limited memory machines: These AI systems can learn from past experiences and make decisions based on that learning. However, their memory is limited, and they cannot generalize their learning to new situat

In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("climate_fever")
df = pd.DataFrame(dataset['test'])
df = df.sample(frac = 1)
df.head()

,claim_id,claim,claim_label,evidences
277,615,If warming temperatures brought on by greenhou...,0,[{'evidence_id': 'Effects of global warming:10...
1272,2568,"""The IPCC also made false predictions on the A...",0,[{'evidence_id': 'Criticism of the IPCC Fourth...
69,141,"Greenpeace didn’t save the whales, switching f...",2,"[{'evidence_id': 'Greenpeace:124', 'evidence_l..."
558,1230,"On the Pacific Coast, a climate pattern that h...",2,"[{'evidence_id': 'California:19', 'evidence_la..."
1455,2951,The two most cited composites are PMOD and ACRIM.,2,"[{'evidence_id': 'Catalan pseudoprime:0', 'evi..."


In [ ]:
def generate():
  generated_dataset = []
  size = 7
  for i in range(size):
      support_prompt = df.sample(n=size)
      prompt_entries = ""
      for index, row in support_prompt.iterrows():
          for evidence in row['evidences']:
              if (evidence['votes'].count("SUPPORTS") >= 2):
                  prompt_entries += ("Claim: " + row['claim']  + " Evidence: " + evidence['evidence'] + " Claim Label: Supports\n")
                  break
              elif (evidence['votes'].count("REFUTES") >= 2):
                  prompt_entries += ("Claim: " + row['claim']  + " Evidence: " + evidence['evidence'] + " Claim Label: Refutes\n")
                  break
  prompt = "Can you generate 5 new entries with a claim and evidence pair having a supports or refutes label similar to the below entries which are related to climate change. Make sure that it is factually correct and adheres to the following json format without using any newline characters: {'Claim': '', 'Evidence': '', 'Claim Label' : ''}"
  prompt += '\n' + prompt_entries + '\n'
  return prompt


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df1 = pd.DataFrame(({'val':[]}))
for id in range(70):
  prompt = generate()
  prompt_template=f'''\nA chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {prompt} ASSISTANT:'''
  input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
  output = model.generate(inputs=input_ids, temperature=0.1, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
  txt = tokenizer.decode(output[0])
  df1 = df1.append({'val':txt},ignore_index=True)
  print(id)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


In [ ]:
for i in range(5):
  print(df1['val'].iloc[i])

<s> 
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: Can you generate 5 new entries with a claim and evidence pair having a supports or refutes label similar to the below entries which are related to climate change. Make sure that it is factually correct and adheres to the following json format without using any newline characters: {'Claim': '', 'Evidence': '', 'Claim Label' : ''}
Claim: Humans are too insignificant to affect global climate Evidence: Accumulation of heat-trapping greenhouse gases, mainly being emitted by people burning fossil fuels, is causing global warming. Claim Label: Refutes
Claim: The heatwave we now have in Europe is not something that was expected with just 1C of warming Evidence: Ruse, Bulgaria hit 36.6 °C (97.9 °F) on the 13th making it the warmest spot in Europe. Claim Label: Supports
Claim: The Earth’s climate is changing in response to increasin

In [ ]:
df1.to_csv('res.csv')

In [ ]:
import re

txt = df1['val'].iloc[0]
x = re.search("ASSISTANT:", txt)
x.start()

1947

In [ ]:
txt = txt[1957:]

In [ ]:
print(txt)

 1. Claim: The Earth's climate is not changing as a result of human activities. Evidence: The Earth's climate has been changing naturally for millions of years, and there is no scientific consensus on the extent to which human activities are contributing to current climate change. Claim Label: Refutes
2. Claim: The Arctic sea ice is recovering after a period of decline. Evidence: The Arctic sea ice has been declining at a rate of about 12.8% per decade since 1979, and the extent of the ice in 2021 was the second-lowest on record. Claim Label: Supports
3. Claim: The oceans are becoming more acidic due to increased carbon dioxide emissions. Evidence: The pH of the ocean has decreased by about 0.1 units since the beginning of the Industrial Revolution, and the rate of acidification is accelerating. Claim Label: Supports
4. Claim: The IPCC's reports are politically motivated and do not accurately reflect the state of the science. Evidence: The IPCC is an intergovernmental organization that

In [ ]:
# x = re.search(".*Claim: (?P<claim>.*?)Evidence: (?P<evid>.*?)",txt)
pattern = ".*Claim: ?(.*)?Evidence: ?(.*)?Claim Label:.*(Refutes|Supports)"

# match.group('rating')
# x.group('evid')

for match in re.finditer(pattern, txt):
    print("Group 1:", match.group(1))
    print("Group 2:", match.group(2))
    print("Group 3:", match.group(3))

Group 1: The Earth's climate is not changing as a result of human activities. 
Group 2: The Earth's climate has been changing naturally for millions of years, and there is no scientific consensus on the extent to which human activities are contributing to current climate change. 
Group 3: Refutes
Group 1: The Arctic sea ice is recovering after a period of decline. 
Group 2: The Arctic sea ice has been declining at a rate of about 12.8% per decade since 1979, and the extent of the ice in 2021 was the second-lowest on record. 
Group 3: Supports
Group 1: The oceans are becoming more acidic due to increased carbon dioxide emissions. 
Group 2: The pH of the ocean has decreased by about 0.1 units since the beginning of the Industrial Revolution, and the rate of acidification is accelerating. 
Group 3: Supports
Group 1: The IPCC's reports are politically motivated and do not accurately reflect the state of the science. 
Group 2: The IPCC is an intergovernmental organization that brings togeth

In [ ]:
data = pd.read_csv('res.csv')

In [ ]:
import json

supports = []
refutes = []
for txt in data['val']:
  x = re.search("ASSISTANT:", txt)
  txt = txt[x.end():]
  pattern = ".*Claim: ?(.*)?Evidence: ?(.*)?Claim Label:.*(Refutes|Supports)"
  for match in re.finditer(pattern, txt):
      dic = {
          "Claim": match.group(1),
          "Evidence": match.group(2)
      }
      label = match.group(3)
      if label == "Refutes":
        refutes.append(dic)
      elif label == "Supports":
        supports.append(dic)

In [ ]:
refutes

[{'Claim': "The Earth's climate is not changing as a result of human activities. ",
  'Evidence': "The Earth's climate has been changing naturally for millions of years, and there is no scientific consensus on the extent to which human activities are contributing to current climate change. "},
 {'Claim': "The IPCC's reports are politically motivated and do not accurately reflect the state of the science. ",
  'Evidence': 'The IPCC is an intergovernmental organization that brings together thousands of scientists from around the world to review and synthesize the latest research on climate change. '},
 {'Claim': "The Earth's temperature has not increased significantly since the year 2000. ",
  'Evidence': "The Earth's temperature has continued to rise since the year 2000, with 2020 being the hottest year on record. "},
 {'Claim': "'Climate change is a hoax' is a myth — so say 80 graphs from 58 peer-reviewed scientific papers published in 2017. ",
  'Evidence': 'Delingpole has engaged in 

In [ ]:
# Serializing json
json_object = json.dumps(supports, indent=4)

# Writing to sample.json
with open("supports.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
json_object = json.dumps(refutes, indent=4)

# Writing to sample.json
with open("refutes.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
len(supports)

305

In [ ]:
len(refutes)

26